<img src=images/pyladiesatx.jpeg align="left" width="12%"></div>

<img src=images/spark-logo-hd.png align="right" width="12%"></div>
<img src=images/python-logo-notext.png align="right" width="6%"></div>


<h1 align='center'>Intro to PySpark</h1>
<h3 align='center'>June 20, 2016 -- PyLadies ATX</h1>

# Environment Setup
We'll use the free Databricks Comunity Edition platform to run our Spark jobs: 
1. Use Google Chrome browser (Firefox should also work, but not Internet Explorer, Safari, etc.)
2. Sign up for the Community Edition account here: https://databricks.com/try-databricks

Or, feel free to use a local installation of Spark, etc. If Spark isn't already installed on your machine it can take up to an hour to download and build from source locally (there are also pre-built versions that would be faster to set up):
1. Download: http://spark.apache.org/downloads.html
2. Open: `../spark1.6.1/README.md`
3. Build: `../spark1.6.1/build/mvn -DskipTests clean package`
    - On my laptop, time to build was ~30 mins.

Q. Before I get started, I'm just curious, if I could see a show of hands, how many of you have used Spark before?  
Q. How many of your are regular Python users?

# Overview
- Spark: What and Why?
- PySpark API
- Examples:
    - Word Count
    - Logistic Regression
    - Clickstream
- References and Resources

# Spark: What and Why

So I'll just briefly give an introduction to what is Spark, for those of you who may be new to it.

Spark is a fast and expressive cluster computing system for doing Big Data computation. It's good for iterative tasks, for doing big batch processing, and for interactive data exploration. 

And it's compatible with Hadoop-supported file systems and data formats (HDFS, S3, SequenceFile, ...), so you can use it with your existing data and deploy it on your existing clusters.

Spark improves efficiency through in-memory computing primitives and general computation graphs.

Spark improves usability through rich APIs in Scala, Python, and Java, and an interactive shell.

## What is Spark?
In contrast to distributed shared memory systems where if you want fault taulerance you have to checkpoint the memory and roll back. RDDs - if you lost a partition of it you can reconstruct it through lineage.

Especially helpful for iterative algorithms
- linear regression
- logistic regression

RDDs do not need to be materialized at all time - they're lazily computed.

Programmers can control 2 aspects of RDDs: caching and partitioning


>"Although current frameworks provide numerous abstractions for accessing a cluster’s computational resources, they lack abstractions for leveraging distributed memory. This makes them inefficient for an important class of emerging applications: those that reuse intermediate results across multiple computations. Data reuse is common in many <font color='blue'>iterative</font> machine learning and graph algorithms, including PageRank, K-means clustering, and logistic regression. Another compelling use case is <font color='blue'>interactive</font> data mining, where a user runs multiple ad-hoc queries on the same subset of the data. Unfortunately, in most current frameworks, the only way to reuse data between computations (e.g., between two MapReduce jobs) is to write it to an external stable storage system, e.g., a distributed file system. This incurs substantial overheads due to data replication, disk I/O, and serialization, which can dominate application execution times."

- Zaharia et al., "Resilient Distributed Datasets: A Fault-Tolerant Abstraction for In-Memory Cluster Computing," *In NSDI '12*, April 2012

## Spark vs MapReduce vs MPI vs ...
- [MapReduce](https://en.wikipedia.org/wiki/MapReduce) --> [Hadoop](http://hadoop.apache.org/): heavily used in business computing
- [Message Passing Interface (MPI)](https://en.wikipedia.org/wiki/Message_Passing_Interface) --> [MVAPICH](http://mvapich.cse.ohio-state.edu/): heavily used in scientific computing
- [Spark](http://spark.apache.org/): good for iterative tasks, big batch processing, interactive data exploration

**Note: The difference between Spark and MapReduce is that you work by partition and not by element. A partition is just a chunk of our data - some number of elements. You can have multiple partitions on one worker node.**

## Spark Architecture
- Spark Driver and Workers
- SparkContext (replaced by SparkSession in version 2.X)

<img src=images/cluster-overview.png align="center" width="50%"></div>

<h4 align='right'>https://spark.apache.org/docs/1.1.0/cluster-overview.html</h4>

## Spark (version 1.X) Programming Concepts
- ****SparkContext****: entry point to Spark functions
    - `parallelize(c, numSlices=None)`
- ****Resilient Distributed Datasets (RDDs)****:
    - Immutable, distributed collections of objects
    - Can be cached in memory for fast reuse
- ****Operations on RDDs****:
    - *Transformations*: define a new RDD (map, join, ...)
    - *Actions*: return or output a result (count, save, ...)

## Spark Data Interfaces (versions 1.X and 2.X)

There are several key interfaces that you should understand when you go to use Spark.

-   ****The Dataset****
    -   The Dataset is Apache Spark's newest distributed collection and can be considered a combination of DataFrames and RDDs. It provides the typed interface that is available in RDDs while providing a lot of conveniences of DataFrames. It will be the core abstraction going forward.
-   ****The DataFrame****
    -   The DataFrame is collection of distributed `Row` types. These provide a flexible interface and are similar in concept to the DataFrames you may be familiar with in python (pandas) as well as in the R language.
-   ****The RDD (Resilient Distributed Dataset)****
    -   Apache Spark's first abstraction was the RDD or Resilient Distributed Dataset. Essentially it is an interface to a sequence of data objects that consist of one or more types that are located across a variety of machines in a cluster. RDD's can be created in a variety of ways and are the "lowest level" API available to the user. While this is the original data structure made available, new users should focus on Datasets as those will be supersets of the current RDD functionality.

*(slide taken from "Introduction to Apache Spark on Databricks" notebook)*

The basic unit of abstraction in Spark to represent your data is an RDD, or a Resilient Distributed Dataset. It's an immutable, partitioned collection of objects.

You can create a bunch of different sources, parallelize your data from a text file - it's this big partitioned collection of objects. And the way that you express your computation is by performing

## Transformations
- Transform one RDD to another RDD
- Yields a new RDD

| Transformation | Description | Type\* |
| :------:  | :-----------: | :-----: |
| `map(func)`     | Apply a function over each element | Narrow |
| `flatMap(func)` | Map then flatten output | Narrow |
| `filter(func)`  | Keep only elements where function is `True` | Narrow |
| `sample(withReplacement, fraction, seed)` | Return a sampled subset of this RDD. | Narrow |
| `groupByKey(k, v)` | extension to be used for dest files. | Wide |
| `reduceByKey(func)` | extension to be used for dest files. | Wide |

\* **Narrow** transformations are local to each node and don't imply transfering information across the network. **Wide** transformations

<img src=images/narrow_wide_transformations.png align="center" width="50%"></div>

<h4 align='right'>https://dzone.com/articles/big-data-processing-spark</h4>

## Actions
- Return or output a result

| Action | Description | Try it Out\*|
| :------:  | :-----------:| :---: |
| `collect()`     | Return a list that contains all of the elements in this RDD. | `sc.parallelize([0, 2, 3, 4, 6], 5).glom().collect()` |
| `count()`  | Return the number of elements in this RDD. | `sc.parallelize([2, 3, 4]).count()` |
| `saveAsTextFile(path)` | Save this RDD as a text file, using string representations of elements. | `sc.parallelize(['', 'foo', '', 'bar', ''])\ .saveAsTextFile("/FileStore/foo-bar.txt")])`|
| `first()`    | Return the first element in this RDD. | `sc.parallelize([2, 3, 4]).first()` |
| `take(num)`    | Take the first num elements of the RDD. | `sc.parallelize([2, 3, 4, 5, 6]).take(2)` |

\* Let's try some simple jobs:
1. Go to your databricks Workspace and create a new directory within your Users directory called "2016-06-20-pyladies-pyspark" 
2. Create a notebook called "0-Introduction"  within this directory
3. Type or copy/paste lines of code into separate cells and run them (you will be prompted to launch a cluster) 

Try a few more with transformations *and* actions:

In [ ]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.groupByKey().mapValues(len).collect())
sorted(rdd.groupByKey().mapValues(list).collect())

In [ ]:
from operator import add

rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.reduceByKey(add).collect())

### Example: Log Mining

In [ ]:
lines = sc.textFile("hdfs://...")
errors = lines.filter(_.startsWith("ERROR"))
messages = errors.map(_.split('\t')(2))

messages.filter(_.contains("foo")).count

The computation is expressed declaratively and nothing actually takes place until calling `count` at the end.

# PySpark API
We'll focus on learning the main API calls needed for running Spark jobs through examples.

## What is PySpark?
- Write Spark jobs in Python
- Run interactive jobs in the shell
- Supports C extensions

### Core classes:
#### pyspark.SparkContext

Main entry point for Spark functionality.

#### pyspark.RDD

A Resilient Distributed Dataset (RDD), the basic abstraction in Spark.

#### pyspark.streaming.StreamingContext

Main entry point for Spark Streaming functionality.

#### pyspark.streaming.DStream

A Discretized Stream (DStream), the basic abstraction in Spark Streaming.

#### pyspark.sql.SQLContext

Main entry point for DataFrame and SQL functionality.

#### pyspark.sql.DataFrame

A distributed collection of data grouped into named columns.



## Why use PySpark?
- If you already know Python
- Can use Spark in tandem with your favorite Python libraries
- If you don't need Python libraries, maybe just write code in Scala

# Examples

### Example 1: Word Count

In [ ]:
from pyspark.context import SparkContext

sc = SparkContext(...)
lines = sc.textFile(sys.argv[2], 1)
counts = lines.flatMap(lambda x: x.split(' ') \
                      .map(lambda x: (x, 1)) \
                      .reduceByKey(lambda x, y: x + y))

for (word, count) in counts.collect():
    print "%s : %i" % (word, count)

In [ ]:
# Get initial RDD from the context
file = spark.textFile("hdfs://...")
# Three consecutive transformation of the RDD
counts = file.flatMap(lambda line: line.split(" "))
             .map(lambda word: (word, 1))
             .reduceByKey(lambda a, b: a + b)
# Materialize the RDD using an action
counts.saveAsTextFile("hdfs://...")

### Example 2: Logistic Regression

In [ ]:
# Every record of this DataFrame contains the label and
# features represented by a vector.
df = sqlContext.createDataFrame(data, ["label", "features"])

# Set parameters for the algorithm.
# Here, we limit the number of iterations to 10.
lr = LogisticRegression(maxIter=10)

# Fit the model to the data.
model = lr.fit(df)

# Given a dataset, predict each point's label, and show the results.
model.transform(df).show()

### Example 3: Clickstream

# Resources and References

#### A related Meetup this Thursday:
>`"Invited Speaker Series - Cody Koeninger: Fundamentals of Spark and Kafka"`  
>`Thursday, June 23, 2016`  
>`6:30 PM to 8:45 PM`  
http://www.meetup.com/Austin-ACM-SIGKDD/events/231377005/  

#### MOOCs:
- "Data Science and Engineering with Apache Spark" Series: https://www.edx.org/course/introduction-apache-spark-uc-berkeleyx-cs105x
- "Hadoop Platform and Application Framework": https://www.coursera.org/learn/hadoop/home/week/5

#### Other:
- http://spark.apache.org/docs/latest/api/python/
- http://spark.apache.org/research.html
- http://spark.apache.org/examples.html
- http://go.databricks.com/apache-spark-2.0-presented-by-databricks-co-founder-reynold-xin
- https://dzone.com/articles/big-data-processing-spark

# Thanks for Coming!

Reach out to Meghann Agarwal with any questions or comments on this talk.